Loading the dataset

In [96]:
import json

data_file = open("./training_data_set.json")
raw_data = json.load(data_file)
data_file.close()

sites = raw_data.get("sitesAndTags", [])
print("Quantidade total de sites:", len(sites))

Quantidade total de sites: 120


Treating raw data for training models.
First, we need to actually retrieve the content present in the url.
Then, we strip the content of its HTML and style tags, as they are irrelevant to our classification. For this, we use BeautifulSoup decompose function.

In [97]:
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup

# Function to remove tags
def remove_tags(html):
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        data.decompose()
  
    return ' '.join(soup.stripped_strings)

sites_contents = []
y_sites = []

for [url, usefulness] in sites:
    r = requests.get(url)
    filtered_content = remove_tags(r.content)
    sites_contents.append(filtered_content)
    y_sites.append(usefulness)
    print(url)

https://www.bigboygames.com.br/jogo-fallen-legion-revenants-vanguard-edition-nintendo-switch
https://www.bigboygames.com.br/world-of-warriors-ps4-4955-p993556
https://www.bigboygames.com.br/jogo-monster-hunter-world-seminovo-ps4-11807-br-p1006227
https://www.bigboygames.com.br/jogo-back-4-blood-xbox-one
https://www.bigboygames.com.br/jogo-far-cry-6-xbox-one-p1006787
https://www.bigboygames.com.br/controle-baseus-sem-fio-transparente-seminovo-nintendo-switch
https://www.bigboygames.com.br/cartao-xbox-live-brasil-r200-5598-p995690
https://www.bigboygames.com.br/case-zelda-botw-seminovo-nintendo-switch-lite-15816
https://www.bigboygames.com.br/console-nintendo-switch-lite-amarelo-seminovo-16101
https://www.bigboygames.com.br/case-protetora-nintendo-swtich-lite-cinza-amarelo-14940
https://www.ibyte.com.br/jogo-hades-xbox/p
https://www.ibyte.com.br/marvel-s-spider-man-miles-morales-ps5/p
https://www.ibyte.com.br/game-fifa-18-xbox-one/p
https://www.shockgames.com.br/produto/horizon-zero-dawn

Lastly, we implement feature selection. For this kind of data, that is, documents composed of many words, using bag of words is a sensible approach.
It vectorizes whole documents by storing the amount of times each unique word in the vocabulary has appeared on it, using a very efficient data structure for spending less memory (sparse matrices from scipy).
Even more, we will eliminate portuguese stopwords from the documents, and limit the bag of words' max features to be 500.

In [98]:
%pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to /home/alps2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words=stopwords.words('portuguese'))
X_sites = count_vect.fit_transform(sites_contents)
X_sites.shape

(120, 6956)

Now, it's a good idea to normalize this vector. As it stands, it counts the absolute number of occurrences. But some documents may be longer than others and this fact can cause inaccuracies down the line. We will transform the occurrences count into a frequency measure (term frequency - tf).

In [100]:
from sklearn.feature_extraction.text import TfidfTransformer

# We downscale the weight of too frequent words by turning the use_idf to True.
tf_transformer = TfidfTransformer(use_idf=False).fit(X_sites)
X_sites_tf = tf_transformer.transform(X_sites)
X_sites_tf.shape

(120, 6956)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X_sites_tf, y_sites, random_state=1337)
X_train = X_train.toarray()
X_test = X_test.toarray()

print("Quantidade de sites para treino:", len(X_train))
print("Quantidade de sites para teste:", len(X_test))

Quantidade de sites para treino: 90
Quantidade de sites para teste: 30



If more sites need to be added to the list of bag of words, We need to call transform instead of fit_transform with count_vect.
The following function should prove useful for this.

In [102]:
def generate_bow_list_from_urls(url_list: list):
    """
    Function to get the content and process it properly for predictions.
    If training is True, call fit_transform to generate
    """
    contents_list = []

    for url in url_list:
        r = requests.get(url)
        filtered_content = remove_tags(r.content)
        contents_list.append(filtered_content)
    
    filtered_contents_bow = count_vect.transform(contents_list)
    filtered_contents_bow_tf = tf_transformer.transform(filtered_contents_bow)
    return filtered_contents_bow_tf

Creating a holding dict for all models predictions.

In [103]:
predictions = {}

Now we need to choose a model and fit it with the training set.
There are many choices here. We will start with basic gaussian naive bayes, and change later to different models to see how they fare.

In [104]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions['gaussian_naive_bayes'] = gnb.predict(X_test)

Now testing with Multinomial Naive Bayes.

In [105]:
from sklearn.naive_bayes import MultinomialNB

mnnb = MultinomialNB()
mnnb.fit(X_train, y_train)
predictions['multinomial_naive_bayes'] = mnnb.predict(X_test)

Now testing with decision tree.

In [106]:
from sklearn.tree import DecisionTreeClassifier
dt =  DecisionTreeClassifier()
dt.fit(X_train, y_train)
predictions['decision_tree'] = dt.predict(X_test)

Calculating accuraccy, recall and f1 score of models

In [107]:
from sklearn.metrics import precision_recall_fscore_support

for model_name in predictions:
    print("Stats for", model_name)

    pred = predictions[model_name]
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=[True, False])
    print("Precision:", precision)
    print("recall:", recall)
    print("f1:", f1)
    print("===============================================================")

Stats for gaussian_naive_bayes
Precision: [0.8 0.6]
recall: [0.66666667 0.75      ]
f1: [0.72727273 0.66666667]
Stats for multinomial_naive_bayes
Precision: [0.47058824 0.23076923]
recall: [0.44444444 0.25      ]
f1: [0.45714286 0.24      ]
Stats for decision_tree
Precision: [0.83333333 0.55555556]
recall: [0.55555556 0.83333333]
f1: [0.66666667 0.66666667]


Consideramos que para o nosso classificador, a precisão é mais relevante que o recall. Basicamente, a precisão é a capacidade do modelo de não marcar um exemplo negativo como positivo. E Recall é capacidade de classificar corretamente todos os exemplos positivos. Para um web-scrapper, deixar de marcar alguns sites corretos como tal não é uma perda tão grande, mas marcar sites incorretos, é.